<a href="https://colab.research.google.com/github/Jang-KyungWuk/KR_Community_Word_Cloud_Generator/blob/master/Ruliweb%20Keyword%20Analyze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import Drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
# KoNLPy Install
!apt-get update
!apt-get install g++ openjdk-8-jdk 
!pip3 install konlpy

# Install KR Font
!apt -qq -y install fonts-nanum
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'

# Install emoji to delete emojis
!pip install emoji

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-ba

In [ ]:
# Variables
board_id="300143"
'''
북유게        "300148"
유게          "300143"
유머게시판    "300143?cate=497"  
라스트 오리진 "184992"
명일방주      "185062"
'''

crawl_min=1
crawl_max=101

crawl_title=True
crawl_article=False

img_file="Common Bottlenose Dolphin_b.png"
base_dir="/content/gdrive/My Drive/"
img_dir=base_dir+img_file

user_dic_name="user_dic.txt"
show_tags=["NNG","NNP","NR","IC","SL","SH","SW","SN"]

In [ ]:
# Import libraries

# For web crawling
from bs4 import BeautifulSoup
import requests as rq
import re

# For naming
import datetime

# For word cloud generating
import numpy as np
import random
from PIL import Image
from wordcloud import WordCloud, STOPWORDS
from palettable.colorbrewer.qualitative import Dark2_8

# For kr analysis
import konlpy
import nltk

# For emoji delete
import emoji

In [ ]:
# Get current time (KST = GMT +9)
KST=datetime.timezone(datetime.timedelta(hours=9))
now=datetime.datetime.now(KST)
MM=now.month
DD=now.day
hh=now.hour
mm=now.minute
c_time=("%s-%s-%s-%s")%(MM,DD,hh,mm)

In [ ]:
# Base URL
main_url="https://bbs.ruliweb.com/community/board"
base_url=main_url*1
base_url+=(("/%s?page=")%board_id)

In [ ]:
# File names, directories
file_id=("%s %s~%s")%(c_time,crawl_min,(crawl_max-1))

title_file_name=("%s title crawl %s.txt")%(board_id, file_id)
title_file_dir=base_dir+title_file_name

article_file_name=("%s article crawl %s.txt")%(board_id, file_id)
article_file_dir=base_dir+article_file_name

bbs_file_name=("%s bbs crawl %s.txt")%(board_id, file_id)
bbs_file_dir=base_dir+bbs_file_name

In [ ]:
# Open txt files to save data
if crawl_title==True:
  title_crawl_txt=open(title_file_dir,'w')

if crawl_article==True:
  bbs_crawl_txt=open(bbs_file_dir,'w')

# Crawl titles and relative directory of bbs articles by moving bbs pages
pages=range(crawl_min,crawl_max)
for page in pages:
  url=base_url+str(page)
  html=rq.get(url, headers={'User-Agent':'el anemos keyword analyzer for word cloud'}).text
  soup=BeautifulSoup(html,'lxml')
  title_soup=soup.find_all('a', attrs={'class':'deco'})
  title_tags=title_soup[len(title_soup)-32 : len(title_soup)-4]
  for t in title_tags:
    if crawl_title==True:
      title_crawl_txt.write(t.text+'\n')
    if crawl_article==True:
      bbs_crawl_txt.write(t['href']+'\n')

# Close txt files
if crawl_title==True:
  title_crawl_txt.close()
  
if crawl_article==True:
  bbs_crawl_txt.close()

In [ ]:
# Crawl articles by movind article pages
if crawl_article==True:
  bbs_crawl_txt=open(bbs_file_dir,'r')
  article_crawl_txt=open(article_file_dir,'w')
  
  dirs=np.loadtxt(bbs_file_dir,delimiter='\n',dtype="str")
  for dir in dirs:
    html=rq.get(dir, headers={'User-Agent':'el anemos keyword analyzer for word cloud'}).text
    soup=BeautifulSoup(html,'lxml')
    article_tags=soup.find_all("div",class_="view_content")
    article_text=''
    for article_tag in article_tags:
      article_text+=(article_tag.text+' ')
    article_text=article_text.replace('\u200b',' ')
    article_crawl_txt.write(article_text+'\n')

  bbs_crawl_txt.close()
  article_crawl_txt.close()

In [ ]:
# Function for color
def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
  return tuple(Dark2_8.colors[random.randint(0,7)])

In [ ]:
# str type value for save all crawled materials
wordseed=""

if crawl_title==True:
  title_crawl_txt=open(title_file_dir,'r')
  wordseed+=title_crawl_txt.read()
  title_crawl_txt.close()

if crawl_article==True:
  article_crawl_txt=open(article_file_dir,'r')
  wordseed+=article_crawl_txt.read()
  article_crawl_txt.close()

wordseed=wordseed.replace('\n',' ').replace('\u3000',' ')

In [ ]:
# Delete Emoji
wordseed=emoji.get_emoji_regexp().sub(u'',wordseed)

In [ ]:
tagged_words=konlpy.tag.Komoran(userdic=base_dir+user_dic_name).pos(wordseed)
grammar="""
NP: {<N.*>*<Suffix>?}   # Noun phrase
VP: {<V.*>*}            # Verb phrase
AP: {<A.*>*}            # Adjective phrase
"""
parser=nltk.RegexpParser(grammar)

In [ ]:
wc_words=""
for morpheme in tagged_words:
  if morpheme[1] in show_tags:
    wc_words+=("%s "%morpheme[0]) 

In [ ]:
# Word cloud generate (Komoran+user_dic)
icon=Image.open(img_dir)
mask=Image.new("RGB",icon.size, (255,255,255))
mask.paste(icon,icon)
mask=np.array(mask)

wc=WordCloud(font_path=fontpath, background_color="white",max_words=300,mask=mask,max_font_size=300,random_state=1234567)

wc.generate_from_text(wc_words)
wc.recolor(color_func=color_func, random_state=1234567)
wc.to_file(base_dir+("%s Word Cloud %s.png")%(board_id,file_id))